In [ ]:
import os

#split data into train test 

image_path = '/content/drive/MyDrive/data/VOC/jpegimages'

all_file_list = [file.replace('.png','') for file in os.listdir(image_path)]

print(len(all_file_list))

134


In [ ]:
import random

all_file_list = random.sample(all_file_list, len(all_file_list))

base_path = '/content/drive/MyDrive/data'

train_images = 'train.txt'
test_images = 'test.txt'
val_images = 'val.txt'

with open(os.path.join(base_path,train_images),'w') as f:
    for item in all_file_list[:100]:
        f.writelines(item+'\n')


with open(os.path.join(base_path,test_images),'w') as f:
    for item in all_file_list[100:120]:
        f.writelines(item+'\n')


with open(os.path.join(base_path,val_images),'w') as f:
    for item in all_file_list[120:]:
        f.writelines(item+'\n')



In [ ]:
import os
import glob
import pandas as pd
import xml.etree.ElementTree as ET

base_path = '/content/drive/MyDrive/data'
anno_path = os.path.join(base_path, "VOC/annotation")
classes_names = []
xml_list = []
print(anno_path)
for xml_file in glob.glob(anno_path + "/*.xml"):
    tree = ET.parse(xml_file)
    root = tree.getroot()
    for member in root.findall("object"):
        classes_names.append(member[0].text)
classes_names = list(set(classes_names))
classes_names.sort()
classes_names

/content/drive/MyDrive/data/VOC/annotation


['station_names', 'table']

In [ ]:
import sys
import os
import json
import xml.etree.ElementTree as ET
import glob

START_BOUNDING_BOX_ID = 1
PRE_DEFINE_CATEGORIES = None
# If necessary, pre-define category and its id
#  PRE_DEFINE_CATEGORIES = {"aeroplane": 1, "bicycle": 2, "bird": 3, "boat": 4,
#  "bottle":5, "bus": 6, "car": 7, "cat": 8, "chair": 9,
#  "cow": 10, "diningtable": 11, "dog": 12, "horse": 13,
#  "motorbike": 14, "person": 15, "pottedplant": 16,
#  "sheep": 17, "sofa": 18, "train": 19, "tvmonitor": 20}


def get(root, name):
    vars = root.findall(name)
    return vars


def get_and_check(root, name, length):
    vars = root.findall(name)
    if len(vars) == 0:
        raise ValueError("Can not find %s in %s." % (name, root.tag))
    if length > 0 and len(vars) != length:
        raise ValueError(
            "The size of %s is supposed to be %d, but is %d."
            % (name, length, len(vars))
        )
    if length == 1:
        vars = vars[0]
    return vars


def get_filename_as_int(filename):
    try:
        filename = filename.replace("\\", "/").replace('page','')
        filename = os.path.splitext(os.path.basename(filename))[0]
        return int(filename)
    except:
        raise ValueError("Filename %s is supposed to be an integer." % (filename))


def get_categories(xml_files):
    """Generate category name to id mapping from a list of xml files.
    
    Arguments:
        xml_files {list} -- A list of xml file paths.
    
    Returns:
        dict -- category name to id mapping.
    """
    classes_names = []
    for xml_file in xml_files:
        tree = ET.parse(xml_file)
        root = tree.getroot()
        for member in root.findall("object"):
            classes_names.append(member[0].text)
    classes_names = list(set(classes_names))
    classes_names.sort()
    return {name: i for i, name in enumerate(classes_names)}


def convert(xml_files, json_file):
    json_dict = {"images": [], "type": "instances", "annotations": [], "categories": []}
    if PRE_DEFINE_CATEGORIES is not None:
        categories = PRE_DEFINE_CATEGORIES
    else:
        categories = get_categories(xml_files)
    bnd_id = START_BOUNDING_BOX_ID
    for xml_file in xml_files:
        tree = ET.parse(xml_file)
        root = tree.getroot()
        path = get(root, "path")
        if len(path) == 1:
            filename = os.path.basename(path[0].text)
        elif len(path) == 0:
            filename = get_and_check(root, "filename", 1).text
        else:
            raise ValueError("%d paths found in %s" % (len(path), xml_file))
        ## The filename must be a number
        image_id = get_filename_as_int(filename)
        size = get_and_check(root, "size", 1)
        width = int(get_and_check(size, "width", 1).text)
        height = int(get_and_check(size, "height", 1).text)
        image = {
            "file_name": os.path.basename(filename.replace("\\","/")),
            "height": height,
            "width": width,
            "id": image_id,
        }
        json_dict["images"].append(image)
        ## Currently we do not support segmentation.
        #  segmented = get_and_check(root, 'segmented', 1).text
        #  assert segmented == '0'
        for obj in get(root, "object"):
            category = get_and_check(obj, "name", 1).text
            if category not in categories:
                new_id = len(categories)
                categories[category] = new_id
            category_id = categories[category]
            bndbox = get_and_check(obj, "bndbox", 1)
            xmin = int(get_and_check(bndbox, "xmin", 1).text) - 1
            ymin = int(get_and_check(bndbox, "ymin", 1).text) - 1
            xmax = int(get_and_check(bndbox, "xmax", 1).text)
            ymax = int(get_and_check(bndbox, "ymax", 1).text)
            assert xmax > xmin
            assert ymax > ymin
            o_width = abs(xmax - xmin)
            o_height = abs(ymax - ymin)
            ann = {
                "area": o_width * o_height,
                "iscrowd": 0,
                "image_id": image_id,
                "bbox": [xmin, ymin, o_width, o_height],
                "category_id": category_id,
                "id": bnd_id,
                "ignore": 0,
                "segmentation": [],
            }
            json_dict["annotations"].append(ann)
            bnd_id = bnd_id + 1

    for cate, cid in categories.items():
        cat = {"supercategory": "none", "id": cid, "name": cate}
        json_dict["categories"].append(cat)

    os.makedirs(os.path.dirname(json_file), exist_ok=True)
    json_fp = open(json_file, "w")
    json_str = json.dumps(json_dict)
    json_fp.write(json_str)
    json_fp.close()

In [ ]:
xml_dir = base_path+"/VOC/annotation"
xml_files = glob.glob(os.path.join(xml_dir, "*.xml"))

train_split_file = "/content/drive/MyDrive/data/train.txt"
test_split_file = "/content/drive/MyDrive/data/test.txt"
val_split_file = "/content/drive/MyDrive/data/val.txt"

def get_split_xml_files(xml_files, split_file):
    with open(split_file) as f:
        file_names = [line.strip() for line in f.readlines()]
    return [xml_file for xml_file in xml_files if os.path.splitext(os.path.basename(xml_file))[0] in file_names]

train_xml_files = get_split_xml_files(xml_files, train_split_file)
test_xml_files = get_split_xml_files(xml_files, test_split_file)
val_xml_files = get_split_xml_files(xml_files, val_split_file)

convert(train_xml_files, base_path + "/coco/train.json")
convert(test_xml_files, base_path + "/coco/test.json")
convert(val_xml_files, base_path + "/coco/val.json")

In [ ]:
# install dependencies: (use cu101 because colab has CUDA 10.1)
!pip install -U torch==1.5.1+cu101 torchvision==0.6.1+cu101 -f https://download.pytorch.org/whl/torch_stable.html

# install mmcv-full thus we could use CUDA operators
!pip install mmcv-full

# install Pillow 7.0.0 back in order to avoid bug in colab
!pip install Pillow==7.0.0

!pip install mmdet

Looking in links: https://download.pytorch.org/whl/torch_stable.html
     |████████████████████████████████| 704.4MB 27kB/s 
     |████████████████████████████████| 6.7MB 32.2MB/s 
ERROR: torchtext 0.9.1 has requirement torch==1.8.1, but you'll have torch 1.5.1+cu101 which is incompatible.
  Found existing installation: torch 1.8.1+cu101
    Uninstalling torch-1.8.1+cu101:
      Successfully uninstalled torch-1.8.1+cu101
  Found existing installation: torchvision 0.9.1+cu101
    Uninstalling torchvision-0.9.1+cu101:
      Successfully uninstalled torchvision-0.9.1+cu101
     |████████████████████████████████| 307kB 18.8MB/s 
     |████████████████████████████████| 194kB 41.5MB/s 
  Created wheel for mmcv-full: filename=mmcv_full-1.3.5-cp37-cp37m-linux_x86_64.whl size=25617465 sha256=4e1580d05a9e8f0dc0dc1cc3a7cbee92651b5fe00628ff87f575ecdf43c7748c
  Stored in directory: /root/.cache/pip/wheels/06/1b/49/7a6ca2a423aa7ad9cbd2caf83f10e0ee09aa06e109994dcc0b
Successfully built mmcv-full
     

     |████████████████████████████████| 593kB 27.5MB/s 
  Created wheel for terminaltables: filename=terminaltables-3.1.0-cp37-none-any.whl size=15356 sha256=bc6a3988d409a170dff66accc2e18ee1b2c8bb718eaf34f93cab4eb81df1011b
  Stored in directory: /root/.cache/pip/wheels/30/6b/50/6c75775b681fb36cdfac7f19799888ef9d8813aff9e379663e
Successfully built terminaltables


In [ ]:
# Install mmdetection
!rm -rf mmdetection
!git clone https://github.com/open-mmlab/mmdetection.git
%cd mmdetection

!pip install -e .

Cloning into 'mmdetection'...
remote: Enumerating objects: 18028, done.
remote: Counting objects: 100% (124/124), done.
remote: Compressing objects: 100% (101/101), done.
remote: Total 18028 (delta 45), reused 67 (delta 23), pack-reused 17904
Receiving objects: 100% (18028/18028), 21.40 MiB | 33.87 MiB/s, done.
Resolving deltas: 100% (12521/12521), done.
/content/mmdetection
Obtaining file:///content/mmdetection
  Found existing installation: mmdet 2.12.0
    Uninstalling mmdet-2.12.0:
      Successfully uninstalled mmdet-2.12.0
  Running setup.py develop for mmdet


In [ ]:
# Check Pytorch installation
import torch, torchvision
print(torch.__version__, torch.cuda.is_available())

# Check MMDetection installation±
import mmdet
print(mmdet.__version__)

# Check mmcv installation
from mmcv.ops import get_compiling_cuda_version, get_compiler_version
print(get_compiling_cuda_version())
print(get_compiler_version())

1.5.1+cu101 True
2.12.0
11.0
GCC 7.5


In [ ]:
from mmcv import Config
from mmdet.apis import set_random_seed
from mmdet.datasets import build_dataset
from mmdet.models import build_detector
from mmdet.apis import train_detector, init_detector, inference_detector, show_result_pyplot
import torch 

In [ ]:
!gdown "https://drive.google.com/u/0/uc?id=1-QieHkR1Q7CXuBu4fp3rYrvDG9j26eFT"

Downloading...
From: https://drive.google.com/u/0/uc?id=1-QieHkR1Q7CXuBu4fp3rYrvDG9j26eFT
To: /content/epoch_36.pth
664MB [00:07, 90.3MB/s]


In [ ]:
cfg = Config.fromfile('/content/drive/MyDrive/data/Training_config.py')
cfg.dataset_type = 'CocoDataset'
cfg.classes = ("station_names", "table",)
# number of classes

cfg.model.roi_head.bbox_head[0].num_classes = 2
cfg.model.roi_head.bbox_head[1].num_classes = 2
cfg.model.roi_head.bbox_head[2].num_classes = 2


In [ ]:
PREFIX = 'VOC/jpegimages'


cfg.data.train.img_prefix = PREFIX
cfg.data.train.data_root = '/content/drive/MyDrive/data'
cfg.data.train.ann_file = 'coco/train.json'
cfg.data.train.classes = cfg.classes
cfg.data.train.type = 'CocoDataset'

cfg.data.val.img_prefix = PREFIX
cfg.data.val.data_root = '/content/drive/MyDrive/data'
cfg.data.val.ann_file = 'coco/val.json'
cfg.data.val.classes = cfg.classes
cfg.data.val.type = 'CocoDataset'

cfg.data.test.img_prefix = PREFIX
cfg.data.test.data_root = '/content/drive/MyDrive/data'
cfg.data.test.ann_file = 'coco/test.json'
cfg.data.test.classes = cfg.classes
cfg.data.test.type = 'CocoDataset'


cfg.load_from = '/content/drive/MyDrive/data/init_model/epoch_36.pth'
cfg.work_dir = "/content/drive/MyDrive/data/work_dir"

cfg.optimizer.lr = 0.02 / 8
cfg.lr_config.warmup = None
cfg.log_config.interval = 600


# We can set the evaluation interval to reduce the evaluation times
cfg.evaluation.interval = 3
cfg.evaluation.classwise = True
# We can set the checkpoint saving interval to reduce the storage cost
cfg.checkpoint_config.interval = 3
# Set seed thus the results are more reproducible
cfg.seed = 0
set_random_seed(0, deterministic=False)
cfg.gpu_ids = range(1)

cfg.lr_config = dict(
    warmup=None,
    warmup_iters=500,
    warmup_ratio=0.001,
    step=[8, 11],
    policy='step')

In [ ]:
print(f'Config:\n{cfg.pretty_text}')

Config:
model = dict(
    type='CascadeRCNN',
    pretrained='open-mmlab://msra/hrnetv2_w32',
    backbone=dict(
        type='HRNet',
        extra=dict(
            stage1=dict(
                num_modules=1,
                num_branches=1,
                block='BOTTLENECK',
                num_blocks=(4, ),
                num_channels=(64, )),
            stage2=dict(
                num_modules=1,
                num_branches=2,
                block='BASIC',
                num_blocks=(4, 4),
                num_channels=(32, 64)),
            stage3=dict(
                num_modules=4,
                num_branches=3,
                block='BASIC',
                num_blocks=(4, 4, 4),
                num_channels=(32, 64, 128)),
            stage4=dict(
                num_modules=3,
                num_branches=4,
                block='BASIC',
                num_blocks=(4, 4, 4, 4),
                num_channels=(32, 64, 128, 256)))),
    neck=dict(type='HRFPN', in_channels=[

In [ ]:
import mmcv
import os.path as osp
# Build dataset
datasets = [build_dataset(cfg.data.train)]

# Build the detector
model = build_detector( cfg.model, train_cfg=cfg.get('train_cfg'), test_cfg=cfg.get('test_cfg'))
# Add an attribute for visualization convenience
model.CLASSES = datasets[0].CLASSES

print(model.CLASSES)
# Create work_dir
mmcv.mkdir_or_exist(osp.abspath(cfg.work_dir))
train_detector(model, datasets, cfg, distributed=False, validate=True)

loading annotations into memory...
Done (t=0.27s)
creating index...
index created!


/content/mmdetection/mmdet/models/backbones/hrnet.py:283: UserWarning: DeprecationWarning: pretrained is a deprecated, please use "init_cfg" instead
  warnings.warn('DeprecationWarning: pretrained is a deprecated, '


('station_names', 'table')
loading annotations into memory...


2021-05-30 21:52:32,309 - mmdet - INFO - load checkpoint from /content/drive/MyDrive/data/init_model/epoch_36.pth
2021-05-30 21:52:32,310 - mmdet - INFO - Use load_from_local loader


Done (t=0.25s)
creating index...
index created!


...

2021-05-30 22:09:20,700 - mmdet - INFO - Epoch(val) [15][14]	bbox_mAP: 0.9060, bbox_mAP_50: 1.0000, bbox_mAP_75: 0.9790, bbox_mAP_s: -1.0000, bbox_mAP_m: -1.0000, bbox_mAP_l: 0.9060, bbox_mAP_copypaste: 0.906 1.000 0.979 -1.000 -1.000 0.906


[>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>] 14/14, 4.5 task/s, elapsed: 3s, ETA:     0s

2021-05-30 22:12:28,262 - mmdet - INFO - Evaluating bbox...
2021-05-30 22:12:28,300 - mmdet - INFO - 
+---------------+-------+----------+-------+
| category      | AP    | category | AP    |
+---------------+-------+----------+-------+
| station_names | 0.814 | table    | 1.000 |
+---------------+-------+----------+-------+
2021-05-30 22:12:28,302 - mmdet - INFO - Saving checkpoint at 18 epochs


Loading and preparing results...
DONE (t=0.00s)
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=0.02s).
Accumulating evaluation results...
DONE (t=0.01s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.907
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=1000 ] = 1.000
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=1000 ] = 0.979
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=1000 ] = -1.000
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=1000 ] = -1.000
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= large | maxDets=1000 ] = 0.907
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.921
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=300 ] = 0.921
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=1000 ] = 0.921
 Average Recall     (AR) @[ IoU=0.50:0.95 | area= small | ma

2021-05-30 22:12:36,739 - mmdet - INFO - Epoch(val) [18][14]	bbox_mAP: 0.9070, bbox_mAP_50: 1.0000, bbox_mAP_75: 0.9790, bbox_mAP_s: -1.0000, bbox_mAP_m: -1.0000, bbox_mAP_l: 0.9070, bbox_mAP_copypaste: 0.907 1.000 0.979 -1.000 -1.000 0.907
2021-05-30 22:14:39,597 - mmdet - INFO - Saving checkpoint at 20 epochs


In [ ]:
img = mmcv.imread('/content/drive/MyDrive/data/VOC/jpegimages/120.png')

checkpoint_file = '/content/drive/MyDrive/data/work_dir/latest.pth'

#model.cfg = cfg
# build the model from a config file and a checkpoint file
model = init_detector(cfg, checkpoint_file, device='cuda:0')

result = inference_detector(model, img)
#show_result_pyplot(model, img, result)
model.show_result(img,result, out_file = '/content/drive/MyDrive/data/result1.jpg')

Use load_from_local loader


/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:2973: UserWarning: Default upsampling behavior when mode=bilinear is changed to align_corners=False since 0.4.0. Please specify align_corners=True if the old behavior is desired. See the documentation of nn.Upsample for details.
  "See the documentation of nn.Upsample for details.".format(mode))


In [ ]:
show_result_pyplot(model,img,result)

In [ ]:
from mmdet.apis import init_detector, inference_detector, show_result_pyplot
import mmcv
# Load model
config_file = '/content/drive/MyDrive/data/Training_config.py'
checkpoint_file = '/content/drive/MyDrive/data/work_dir/latest.pth'
model = init_detector(config_file, checkpoint_file, device='cuda:0')

# Test a single image 
img = "/content/drive/MyDrive/data/VOC/jpegimages/16.png"

# Run Inference
result = inference_detector(model, img)

# Visualization results
model.show_result(img,result, out_file = '/content/drive/MyDrive/data/result1.jpg')

In [ ]:
result[0][0]

In [ ]:
res_borderTable = []
res_cell = []

## for tables with borders
for r in result[0]:
    if r[4]>.85:
        res_borderTable.append(r[:4].astype(int))
    ## for cells
for r in result[1]:
    if r[4]>.85:
        res_cell.append(r[:4].astype(int))

In [ ]:
print(res_borderTable)
print(res_cell)

In [ ]:
from google.colab.patches import cv2_imshow
import cv2
import pytesseract


img = cv2.imread("/content/drive/MyDrive/data/VOC/jpegimages/16.png")
#crop = img[171:277, 229:1455]

sn = []
table= []

for i in res_borderTable:
    crop = img[i[1]-10:i[3]+10, i[0]-10:i[2]+10]
    cv2_imshow(crop)
    text = pytesseract.image_to_string(crop, config="--psm 6 --oem 2 tessedit_char_blacklist=1234567890", 
                                       lang='rus+eng')
    sn.append(text)
    print(text)

config = "-c tessedit_char_whitelist=0123456789.()- --oem 3 --psm 6"
for i in res_cell:
    crop = img[i[1]-5:i[3]+5, i[0]-5:i[2]+5]
    cv2_imshow(crop)
    text = pytesseract.image_to_string(crop, config=config, lang='datarescue')
    table.append(text)
    print(text)

In [ ]:
final_text = sn[0]+table[0]+sn[1]+table[1]
print(final_text)

In [ ]:
print(sn[0])

In [ ]:
!pip install pytesseract
!sudo apt-get update
!sudo apt-get install tesseract-ocr

In [ ]:
!wget https://github.com/tesseract-ocr/tessdata/raw/master/rus.traineddata

!sudo mv -v /content/datarescue.traineddata /usr/share/tesseract-ocr/4.00/tessdata/

In [ ]:
!wget https://github.com/tesseract-ocr/tessdata/raw/master/eng.traineddata

!sudo mv -v /content/eng.traineddata /usr/share/tesseract-ocr/4.00/tessdata/

--2021-05-22 21:15:05--  https://github.com/tesseract-ocr/tessdata/raw/master/eng.traineddata
Resolving github.com (github.com)... 140.82.112.4
Connecting to github.com (github.com)|140.82.112.4|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/tesseract-ocr/tessdata/master/eng.traineddata [following]
--2021-05-22 21:15:06--  https://raw.githubusercontent.com/tesseract-ocr/tessdata/master/eng.traineddata
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 23466654 (22M) [application/octet-stream]
Saving to: ‘eng.traineddata’

eng.traineddata     100%[===================>]  22.38M  --.-KB/s    in 0.1s    

2021-05-22 21:15:06 (168 MB/s) - ‘eng.traineddata’ saved [23466654/23466654]

renamed '/content/